In [1]:
import sys
import numpy as np
import random
import os
import math
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import animation
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import importlib
import scripts.theodolite_function as tfu
import scripts.theodolite_utils as ttf
import GPy
import scripts.gp_prediction_utils as GPf
tfu = importlib.reload(tfu)
ttf = importlib.reload(ttf)
GPf = importlib.reload(GPf)

from scipy.interpolate import splprep, splev
from scipy import interpolate
from scipy import spatial
import seaborn as sns

from scipy.spatial.transform import Rotation as R
from IPython.display import HTML
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel)
from stheno import B, Measure, GP, EQ, Delta
import torch

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from wbml.plot import tweak


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


Python older than 3.7 detected. 


In [4]:
# Read rosbag of grand axe and show the trajectory
ttf = importlib.reload(ttf)
tfu = importlib.reload(tfu)
GPf = importlib.reload(GPf)

'''
file = ["/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-50-15.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-53-12.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-58-58.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-14-42-27.bag"]
        # "/media/will/T7/theodolite_bags/20220513_TS_2/2022-05-13-22-35-30.bag",
        # "/media/will/T7/theodolite_bags/20220513_TS_2/2022-05-13-22-51-40.bag"]

output = ["TS/20220717/01/",
          "TS/20220717/02/",
          "TS/20220717/03/",
          "TS/20220717/04/",
          "TS/20220717/05/"]
          # "TS/20220513/06/"]  
'''

file = [
        "/home/norlab/Data/IROS_2022/20220224_TS/2022-02-24-15-34-38.bag",
        "/home/norlab/Data/IROS_2022/20220307_TS/2022-03-07-19-20-06.bag",
#         "/home/norlab/Data/IROS_2022/20220312_TS/2022-03-12-09-45-12.bag",
        "/home/norlab/Data/IROS_2022/20220314_TS/2022-03-14-10-47-49.bag",
        "/home/norlab/Data/IROS_2022/20220316_TS/2022-03-16-19-02-42.bag",
#         "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-10-22-52.bag",
        "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-11-20-05.bag",
#         "/home/norlab/Data/IROS_2022/20220427_TS/2022-04-27-12-12-10_filered.bag",
#         "/home/norlab/Data/IROS_2022/20220505_TS/empty1_2022-05-05-19-14-33.bag",
#         "/home/norlab/Data/IROS_2022/20220505_TS/cones1_2022-05-05-19-25-54.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-10-30-37_filtered.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-11-01-56.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-11-24-28.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-21-49-12_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-22-35-30.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-22-51-40.bag",
#         "/home/norlab/Data/IROS_2022/20220523_TS/cones_filtered_2022-05-23-19-31-32.bag",
#         "/home/norlab/Data/IROS_2022/20220523_TS/constrained_2022-05-23-18-56-16.bag",
#         "/home/norlab/Data/IROS_2022/20220523_TS/empty_filtered_2022-05-23-19-18-35.bag",
#         "/home/norlab/Data/IROS_2022/20220523_TS/other_tunnel_2022-05-23-19-39-40.bag",
        "/home/norlab/Data/IROS_2022/20220525_TS/2022-05-25-11-07-40.bag",
        "/home/norlab/Data/IROS_2022/20220525_TS/2022-05-25-13-32-32.bag",
        "/home/norlab/Data/IROS_2022/20220622_TS/2022-06-22-17-48-25.bag",
        "/home/norlab/Data/IROS_2022/20220622_TS/2022-06-22-18-11-00.bag",
        "/home/norlab/Data/IROS_2022/20220630_TS/2022-06-30-10-32-21.bag",
        "/home/norlab/Data/IROS_2022/20220630_TS/2022-06-30-10-57-07.bag",
        "/home/norlab/Data/IROS_2022/20220711_TS/2022-07-11-15-36-04.bag",
        "/home/norlab/Data/IROS_2022/20220711_TS/2022-07-11-15-50-00.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-50-15.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-53-12.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-58-58.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-14-42-27.bag"
#         "/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-10-31.bag",
#         "/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-11-10.bag",
#         "/home/norlab/Data/IROS_2022/20220717_TS/cones_2022-07-17-20-38-29.bag",
#         "/home/norlab/Data/IROS_2022/20220717_TS/empty_2022-07-17-21-03-14.bag",
#         "/home/norlab/Data/IROS_2022/20220717_TS/calibration_2022-07-17-21-27-28.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-27-58.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-29-00.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-30-49.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-32-47.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-34-15.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-12-37-00.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-13-37-52.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-13-49-07.bag",
#          "/home/norlab/Data/IROS_2022/20220910_TS/2022-09-10-13-58-19.bag"
       ]

output = [
        'TS/20220224/',
        'TS/20220307/',
#         'TS/20220312/',
        'TS/20220314/',
        'TS/20220316/',
#         'TS/20220331-1/',
        'TS/20220331-2/',
#         'TS/20220427/',
#         'TS/20220505_empty/',
#         'TS/20220505_cones/',
        'TS/20220513/01/',
        'TS/20220513/02/',
        'TS/20220513/03/',
        'TS/20220513/04/',
#         'TS/20220513/05/',
        'TS/20220513/06/',
#         'TS/20220523_cones/',
#         'TS/20220523_constrained/',
#         'TS/20220523_empty/',
#         'TS/20220523_other_tunnel/',
        'TS/20220525/01/',
        'TS/20220525/02/',
        'TS/20220622-1/',
        'TS/20220622-2/',
        'TS/20220630-1/',
        'TS/20220630-2/',
        'TS/20220711/01/',
        'TS/20220711/02/',
        'TS/20220715/01/',
        'TS/20220715/02/',
        'TS/20220715/03/',
        'TS/20220715/04/'
#         'TS/20220717/01/',
#         'TS/20220717/02/',
#         'TS/20220717/03/',
#         'TS/20220717/04/',
#         'TS/20220717/05/',
#         'TS/20220910/01/',
#         'TS/20220910/02/',
#         'TS/20220910/03/',
#         'TS/20220910/04/',
#         'TS/20220910/05/',
#         'TS/20220910/06/',
#         'TS/20220910/07/',
#         'TS/20220910/08/',
#         'TS/20220910/09/'
]

parameters = [
    [0,2,1,1,1,0,2],
    [1,2,1,1,1,0,2],
    [0,2,1,1,1,0,6],
    [1,2,1,1,1,0,6]
]

for param in parameters:
    print(param)
    # General parameters 
    if(param[0]==0):
        filtering = False
    if(param[0]==1):
        filtering = True
    thresold_d = param[1]                # m/s
    thresold_a = param[2]                # deg/s
    thresold_e = param[3]                # deg/s
    limit_time_interval = param[4]       # s
    path_output = "./data/prediction/"
    B.epsilon = 1e-8
    Mode = "L"
    limit_search = limit_time_interval
    delta_t = param[5]               # Value to remove points near edge of time intervals
    size_interval = param[6]           # Minimum time size sub-interval 
    save = True

    # GP parameters
    verbose=False
    Number_restart = 100
    noise_GP = 0                # Noise of GP
    variance_GP = 1             # Variance of GP
    lengthscale_GP = 1          # Lengthscale of GP

    save_index_1 = []

    for fname, opath in zip(file,output): 
        if(not filtering):
            path = opath + "raw/"
        else:
            path = opath + "filtered/"

        if(filtering):
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
            index_1_f = ttf.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_2_f = ttf.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_3_f = ttf.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1),len(t2),len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data(fname)
            print(len(t1),len(t2),len(t3))

        time_origin = np.min([t1[0],t2[0],t3[0]])

        start_time = time.time()

        t1 = t1 - np.ones_like(t1)*time_origin
        t2 = t2 - np.ones_like(t2)*time_origin
        t3 = t3 - np.ones_like(t3)*time_origin

        list_interval, list_time = tfu.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = tfu.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []

        for i in tqdm(list_trajectories_split):

            index_1 = np.array([i[0,0],i[1,0]])
            index_2 = np.array([i[0,1],i[1,1]])
            index_3 = np.array([i[0,2],i[1,2]])

            save_index_1.append(index_1)

            if(filtering==True):
                index_1 = GPf.delta_t_function(index_1,t1,delta_t)
                index_2 = GPf.delta_t_function(index_2,t2,delta_t)
                index_3 = GPf.delta_t_function(index_3,t3,delta_t)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if(abs(end-begin)>size_interval and begin<end):

                rate = 10  #Hz
                T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))

                if(Mode == "MGPO" or Mode == "All"):
                    T_MGPO, S_MGPO = GPf.data_training_MGPO(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Training")
                    m = GPf.training_MGPO(Number_restart, verbose, T_MGPO, S_MGPO)
                    print("Prediction")
                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_MGPO, P2_MGPO, P3_MGPO = GPf.unit_prediction_MGPO(i, m)
                        Prediction_1.append(P1_MGPO)
                        Prediction_2.append(P2_MGPO)
                        Prediction_3.append(P3_MGPO)

                if(Mode == "GP" or Mode == "All"):
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_GP(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Training")
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = GPf.training_GP(Number_restart, verbose, T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3)
                    print("Prediction")
                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_GP, P2_GP, P3_GP = GPf.unit_prediction_GP(i, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_GP)
                        Prediction_2.append(P2_GP)
                        Prediction_3.append(P3_GP)

                if(Mode == "L" or Mode == "All"):
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Prediction")
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = GPf.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_GP, P2_GP, P3_GP = GPf.linear_prediction(i, time_origin, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_GP)
                        Prediction_2.append(P2_GP)
                        Prediction_3.append(P3_GP)

                if(Mode=="SGP"):
                    prediction_value = T_prediction_init.numpy()
                    # Prepare data for training
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_GP_stheno(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    # Training for each axis
                    m_X1, v_X1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=X1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y1, v_Y1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Y1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z1, v_Z1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Z1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_X2, v_X2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=X2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y2, v_Y2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Y2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z2, v_Z2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Z2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_X3, v_X3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=X3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y3, v_Y3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Y3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z3, v_Z3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Z3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)

                    # Save prediction at time i
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X1, m_Y1, m_Z1, v_X1, v_Y1, v_Z1):
                        Prediction_1.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                        T_prediction.append(i+time_origin)
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X2, m_Y2, m_Z2, v_X2, v_Y2, v_Z2):
                        Prediction_2.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X3, m_Y3, m_Z3, v_X3, v_Y3, v_Z3):
                        Prediction_3.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))

        stop_time = time.time()
        print(stop_time - start_time)

        print("Interpolation finished !")

        if save:
            if(Mode == "MGPO" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)

                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
            if(Mode == "GP" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)

                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
            if(Mode == "SGP" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)

                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
                tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
            if(Mode == "L" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"

                if save:
                    tfu.Convert_datap_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
                    tfu.Convert_datap_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
                    tfu.Convert_datap_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")

        print("Saved !")

[0, 2, 1, 1, 1, 0, 2]
Number of data for theodolites: [4945 4681 4737]
Bad measures: 558
4945 4681 4737


  2%|▏         | 3/199 [00:00<00:07, 26.50it/s]

Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction


 15%|█▌        | 30/199 [00:00<00:01, 93.48it/s]

Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction
Prediction


...

Number of data for theodolites: [620 622 614]
Bad measures: 7
612 614 604


 33%|███▎      | 2/6 [00:00<00:00, 17.95it/s]

Prediction
Prediction
Prediction
Prediction


100%|██████████| 6/6 [00:00<00:00, 19.35it/s]


0.3208119869232178
Interpolation finished !
Conversion done !
Conversion done !
Conversion done !
Saved !


In [ ]:
ttf = importlib.reload(ttf)
tfu = importlib.reload(tfu)

k = './data/prediction/TS/20220717/02/'
path_type = 'raw/'
path_file_type = 'nf-1-6-1-L_'

trimble_1 = ttf.read_prediction_data_resection_csv_file(k+path_type+path_file_type+"1.csv")
t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data("/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-11-10.bag")

In [ ]:
%matplotlib notebook

save_index_1_arr = np.array(save_index_1)
i = save_index_1_arr[20]

plt.figure()

plt.scatter(t1,tp1[0,:],s =1)
#plt.scatter(t2,tp2[0,:],s =1)

plt.scatter(t1[i[0]:i[1]],tp1[0,i[0]:i[1]], linewidth=5, color= 'black')

#plt.plot(trimble_1[i[0,0]:i[1,0],0],trimble_1[i[0,0]:i[1,0],1], linewidth=5, color= 'black')
#plt.plot(trimble_1[i[0,0]:i[1,0],0],trimble_1[i[0,0]:i[1,0],1], linewidth=5, color= 'black')
plt.scatter(trimble_1[:,0],trimble_1[:,1], linewidth=2, color= 'red')


plt.show()